In [4]:
import os
import numpy as np
from scipy.misc import imread
from scipy.misc import imsave
from scipy.misc import imshow
from scipy.misc import imresize
import cv2

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import Nets as nets
import Gans as gans

import sys
sys.path.insert(0, os.path.abspath('./CyclicGen/'))

In [1]:
def run_gan_cfg(f,s):
    cv2.imwrite('first.png', f)
    cv2.imwrite('second.png', s)
    ##%run CyclicGen/CyclicGen_train_stage1.py --pretrained_model_checkpoint_path=./ckpt/CyclicGen/model --subset=test --batch_size=1
    ##print("test ended ok!")
    #%run CyclicGen/run.py ##--pretrained_model_checkpoint_path=./ckpt/CyclicGen/model --first=./first.png --second=./second.png --out=./out.png
    session = gans.get_model(f.shape[0],f.shape[1])
    print("test run_gan_cfg ended ok!")

In [1]:
def load_vid(vid_path):
    print("Loading:", vid_path)
    cap = cv2.VideoCapture(vid_path)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    vid_arr = np.zeros(shape=(num_frames, 128, 384, 3), dtype="uint8") # 128x384 with 3 color channels

    for i in range(num_frames):
        #if i % (num_frames / 10) == 0:
            #print ("Video loading is {0}% done.".format((i / (num_frames / 10) * 10)))

        ret, frame = cap.read()
        vid_arr[i] = imresize(frame, (128, 384))

    ##test
    print(num_frames,vid_arr.shape)
    return vid_arr, fps

In [4]:
## gany python3 run.py --pretrained_model_checkpoint_path=./ckpt/CyclicGen/model --first=./first.png --second=./second.png --out=./out.png

In [1]:
def double_vid_fps(vid_arr, method='u_net'):
    shps = (6, 128, 384)
    #print(shps)
    if(method == 'u_net'):
        model = nets.get_unet_2(shps) #6 -> 3 chans per frame, 2 frames input
        #todo weights
        weight_path = "./deep-motion/model_weights/weights_unet2_finetune_youtube_100epochs.hdf5"
        model.load_weights(weight_path) #weights for trained network
        #print("weights loaded")
    elif(method == 'gan_cfg'):
        session, plch, prediction = gans.get_model(vid_arr[0].shape[0],vid_arr[0].shape[1])
    # new_vid_arr = np.zeros(shape=(len(vid_arr)*2, 128, 384, 3))
    new_vid_arr = []
    new_vid_arr.append(vid_arr[0])
    for i in range(1, len(vid_arr)):
        if i % (len(vid_arr) / 10) == 0:
            print ("FPS doubling is {0}% done.".format((i / (len(vid_arr) / 10) * 10)))
        
        #print("predicting",vid_arr[i-1].shape)
        if(method == 'u_net'):
            pred = model.predict(np.expand_dims(np.transpose(np.concatenate((vid_arr[i-1], vid_arr[i]), axis=2)/255., (2, 0, 1)), axis=0)) #predict middle frame
            new_vid_arr.append((np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")) ##insert middle frame to new_vid_arr
        elif(method == 'gan_cfg'):
            ## gany python3 run.py --pretrained_model_checkpoint_path=./ckpt/CyclicGen/model --first=./first.png --second=./second.png --out=./out.png
            #run_gan_cfg(vid_arr[i-1], vid_arr[i])
            pred = gans.gan_pred(vid_arr[i-1], vid_arr[i], session, plch, prediction)
            #pred = cv2.imread('./out.png')
            new_vid_arr.append(pred) ##insert middle frame to new_vid_arr
        else:
            print("other")
        #new_vid_arr.append((np.transpose(pred[0], (1, 2, 0))*255).astype("uint8")) ##insert middle frame to new_vid_arr
        new_vid_arr.append(vid_arr[i]) #insert next frame -> first frame for upcoming prediction

    return np.asarray(new_vid_arr)

In [2]:
def save_vid(vid_arr, vid_out_path, fps):
    #todo
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    # fourcc = cv2.VideoWriter_fourcc(*'X264')
    out = cv2.VideoWriter(vid_out_path, fourcc, fps, (384, 128))

    for i in range(len(vid_arr)):
        out.write(vid_arr[i])

In [ ]:
#makes a collage of output videos (original, subsampled, deepmotion and cyclegan)
def combine_vid(original, half, unet, gans, fps_max,fps_n, fps_min,outname, out_dir):
    print(original.shape)
    f = gans.shape[0]
    h_f = half.shape[0]
    w = original.shape[2]
    h = original.shape[1]
    
    o_w = w*2+10
    o_h = h*2+10
    canvas = np.ones((f*2,o_h, o_w,3), dtype=np.uint8)

    canvas[0:f,0:h,0:w,:] = original[0:f]
    canvas[f:2*f,0:h,0:w,:] = original[0:f]
    
    #print(f,h_f, f*2 // h_f)
    k = 0
    for i in range((f*2 // h_f)):
        #print(i)
        canvas[i*h_f:(i+1)*h_f,h+10:,0:w,:] = half
        k=i
        
    #print((k+1)*h_f, 2*f)
    canvas[(k+1)*h_f-1:2*f,h+10:,0:w,:] = half[:(f%h_f):,:,:,:]
    canvas[0:unet.shape[0],0:h,w+10:,:] = unet
    canvas[unet.shape[0]:unet.shape[0]*2,0:h,w+10:,:] = unet
    canvas[0:gans.shape[0],h+10:,w+10:,:] = gans
    canvas[gans.shape[0]:gans.shape[0]*2,h+10:,w+10:,:] = gans
    
    print(canvas.shape, len(canvas), (w,h))
    canvas.astype(np.uint8)

    # font 
    font = cv2.FONT_HERSHEY_SIMPLEX  
    # fontScale 
    fontScale = 0.5
    # Blue color in BGR 
    color = (5, 235, 235) 
    # Line thickness of 2 px 
    thickness = 1
    # Using cv2.putText() method 
    for i in range(len(canvas)):
        canvas[i] = cv2.putText(canvas[i], 'orginal '+str(fps_max)+" fps", (20,20), font, fontScale, color, thickness)
        canvas[i] = cv2.putText(canvas[i], 'subsampled '+str(fps_min)+" fps", (20,h+30), font, fontScale, color, thickness, cv2.LINE_AA)
        canvas[i] = cv2.putText(canvas[i], 'U_NET '+str(fps_n)+" fps", (w+30,20), font, fontScale, color, thickness, cv2.LINE_AA)
        canvas[i] = cv2.putText(canvas[i], 'GAN '+str(fps_n)+" fps", (w+30,h+30), font, fontScale, color, thickness, cv2.LINE_AA)

    cv2.imwrite(out_dir + outname+"frame0.png", canvas[5])
    cv2.imwrite(out_dir + outname+"frame1.png", canvas[15])
    cv2.imwrite(out_dir + outname+"frame2.png", canvas[21])
    cv2.imwrite(out_dir + outname+"frame3.png", canvas[33])
    cv2.imwrite(out_dir + outname+"frame4.png", canvas[43])
    cv2.imwrite(out_dir + outname+"frame5.png", canvas[53])    

    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_dir + outname+".mp4", fourcc, fps_max, (o_w, o_h))
    for i in range(len(canvas)):
        out.write(canvas[i])
    print("canvas saved")
    

In [ ]:
# combine all output videos into one
def combinevid(vids,f, outname, out_dir):
    canvas = np.ones((0,266,778,3), dtype=np.uint16)
    for vid in vids:
        v, fps = load_vid(os.path.join(out_dir, vid))
        if(canvas.shape[0] == 0):
            canvas = np.ones((len(v),266,778,3), dtype=np.uint16)
            canvas = v
        else:
            canvas = np.concatenate([canvas,v])
        
    #canvas.astype(np.uint8)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_dir + outname+".avi", fourcc, f, (canvas.shape[2], canvas.shape[1]))
     
    for i in range(len(canvas)):
        out.write(canvas[i])